## Activity 1: 
* Karger’s algorithm finds a minimum cut based on the number of edges. 

* What if we have weights on the edges. How would you modify Karger’s algorithm (or design a new algorithm) to take into account edge weights?
    * You are not expected to write any code here, but design an algorithm. You can present your algorithm in form of a pseudo-code, paragraph, or video recording.
    * You are expected to make any reasonable assumptions about the problem, to simplify your 
formulation of proposed algorithm.

* What is the objective?
    + the goal is to find a cut that minimizes the sum of the weights of the edges crossing the cut.

* How to select the edge for contraction for this purpose?
    + selecting a particular edge `(u, v)` with probability proportional to its weight relative to the total weight. This means edges with higher weights are more likely to be merged (i.e., impossible to be considered as a cut).

* What happen to the edges between two contracted vertices `u` and `v` and Node `z` which has connection to both contracted nodes ?
    + No change
    +  A parallel edge: The weight of any parallel edge is the sum of the weights of the edges being merged.

* How to evaluate the cut?
    + After the graph is reduced to 2 vertices, calculate the total weight of the edges between the two super-vertices. This represents a cut in the original graph.



`function weighted_random_selection(edges):`
        
        total_weight = sum(weights for (u, v, w) in edges)
        r = random(0, total_weight)
        cumulative_weight = 0
        
        for (u, v, w) in edges:
            cumulative_weight += w
            if cumulative_weight >= r:
                return (u, v)



`function weighted_karger(graph, num_iterations):`
        
        min_cut = infinity
        
        for i in range(num_iterations):
            weighted_edges = get_edges(graph)
            (u, v) = weighted_random_selection(weighted_edges)
            contract_nodes(graph, u, v)
            current_cut = calculate_cut_size(graph)
            
            if current_cut < min_cut:
                min_cut = current_cut
            
            restore_original_graph_state(graph)
        
        return min_cut


## Activity 2: Ford-Fulkerson
* Repeatedly find **augmenting paths** through the **residual graph** and 
* **Augment the flow** until no more augmenting paths can be found
  

Questions
* what is the relationship between min cut and max flow?
  * The capacity of any cut BOUNDs value of any flow, i.e., the cut with the minimum capacity points you to the max flow.  Because
    1. Flow to the cut = Flow to the network
    2. Capacity constraint
    


* How is the augmenting path found via breath-first search?
* Why do we need residual/reverse edges in the residual network?
  * Representing the shrinkage of the flow, i.e., Undo bad augmenting paths that do not lead to a maximum flow
  * Every edge in the original graph has a residual edge with a flow/capacity of 0/0

<img src="pics/ford_fulkerson.png">




In [3]:
from copy import deepcopy
from collections import deque
"""
    A    B
S   C    D    T
    E    F
"""
# Represent the graph
graph = {
    'S': {'A': 4, 'C': 2, 'E': 6},
    'A': {'B': 3, 'D': 6},
    'B': {'T': 4},
    'C': {'D': 3},
    'D': {'T': 4},
    'E': {'F': 10},
    'F': {'C': 3, 'T': 4},
    'T': {}
}

def ford_fulkerson(graph, source, target):
    flow = 0
    original_graph = deepcopy(graph)  # Create a deep copy of the original graph for min-cut determination
    
    while True:
        # search for augmenting path with flow reserve using BFS
        path, reserve = breadth_first_search(graph, source, target)
        
        # If no path is found, break out of the loop
        if not path:
            break

        flow += reserve

        # increase flow along the path; 
        # updates the residual capacities of the edges along the path.
        print('Augmenting Path')
        for v, u in zip(path, path[1:]):
            print(v, u)
            if u in graph[v]:
                graph[v][u] -= reserve # Update forward residual edge
                if graph[v][u] == 0:
                    del graph[v][u] # Remove the edge if its capacity becomes 0
            
            if v not in graph[u]:
                graph[u][v] = 0 # Create reverse edge (u, v) if it doesn't exist
            graph[u][v] += reserve # Update reverse residual edge

    # Find the set of vertices reachable from source in the residual graph
    reachable = set()
    stack = [source]
    while stack:
        vertex = stack.pop()
        reachable.add(vertex)
        for neighbor, capacity in graph[vertex].items():
            if neighbor not in reachable and capacity > 0:
                stack.append(neighbor)

    # Find the min-cut edges
    min_cut = []
    for vertex in reachable:
        for neighbor in original_graph[vertex]:  # Use the original (deep copied) graph here
            if neighbor not in reachable:
                min_cut.append((vertex, neighbor))

    return flow, min_cut

def breadth_first_search(graph, source, target):
    visited = set()
    queue = deque([(source, float('inf'))])
    pred = {source: None}

    while queue:
        v, flow = queue.popleft()
        visited.add(v)
        for u, w in graph[v].items():
            if u not in visited and w > 0:
                pred[u] = v
                min_flow = min(flow, w)
                if u == target:
                    path = []
                    while u:
                        path.append(u)
                        u = pred[u]
                    path.reverse()
                    return path, min_flow
                queue.append((u, min_flow))

    return None, 0

# Implement the Ford-Fulkerson algorithm and find the min-cut
max_flow, min_cut = ford_fulkerson(graph, 'S', 'T')
print(f"Maximum Flow: {max_flow}")
print(f"Min-Cut Edges: {min_cut}")



Augmenting Path
S A
A B
B T
Augmenting Path
S C
C D
D T
Augmenting Path
S C
C D
D T
Augmenting Path
S A
A D
D T
Augmenting Path
S E
E F
F T
Augmenting Path
S E
E F
F C
C D
D T
Maximum Flow: 11
Min-Cut Edges: [('S', 'A'), ('F', 'T'), ('C', 'D')]


In [5]:
# bipanjeet
class FordFulkerson:
    def __init__(self, graph):
        self.graph = graph
        self.visited = []

    def dfs(self, node, parent, min_capacity):
        self.visited.append(node)
        if node == 'T':
            return min_capacity

        for neighbor, capacity in self.graph[node]:
            if neighbor not in self.visited and capacity > 0:
                flow = self.dfs(neighbor, node, min(min_capacity, capacity))
                if flow > 0:
                    self.update_capacity(node, neighbor, flow)
                    return flow
        return 0

    # def update_capacity(self, u, v, flow):
    #     for i, (neighbor, capacity) in enumerate(self.graph[u]):
    #         if neighbor == v:
    #             self.graph[u][i] = (neighbor, capacity - flow)
    #             break
    #     for i, (neighbor, capacity) in enumerate(self.graph[v]):
    #         if neighbor == u:
    #             self.graph[v][i] = (neighbor, capacity + flow)
    #             break

# For the implementation of FordFulkerson,  how do you deal with the case when the reversed edge does not exist in the graph during update_capacity.
# Take an example as below: graph = { 'S': [('A', 11), ('B', 7)], 'A': [('B', 13), ('T', 10)], 'B': [('A', 21), ('T', 14)], 'T': [] }
# A flow of 4 from S to A. But since there is no "S" in self.graph['A'], there is no update of the reversed edge.

    def update_capacity(self, u, v, flow):
        # Update the forward edge (u, v)
        for i, (neighbor, capacity) in enumerate(self.graph[u]):
            if neighbor == v:
                self.graph[u][i] = (neighbor, capacity - flow)
                break

        # Update or create the reverse edge (v, u)
        reverse_edge_exists = False
        for i, (neighbor, capacity) in enumerate(self.graph[v]):
            if neighbor == u:
                self.graph[v][i] = (neighbor, capacity + flow)
                reverse_edge_exists = True
                break

        # If the reverse edge does not exist, add it
        if not reverse_edge_exists:
            self.graph[v].append((u, flow))


    def max_flow(self, source, sink):
        max_flow_value = 0
        while True:
            self.visited.clear()
            flow = self.dfs(source, None, float('inf'))
            if flow == 0:
                break
            max_flow_value += flow

        return max_flow_value

    def min_cut(self, source):
        min_cut_edges = []
        for node in self.visited:
            for neighbor, capacity in self.graph[node]:
                if neighbor not in self.visited:
                    min_cut_edges.append((node, neighbor))
        return min_cut_edges


   
graph = {
    'S': [('A', 11), ('B', 7)],
    'A': [('B', 13), ('T', 10)],
    'B': [('A', 21), ('T', 14)],
    'T': []
}
# Represent the graph
graph = {
    'S': {'A': 4, 'C': 2, 'E': 6},
    'A': {'B': 3, 'D': 6},
    'B': {'T': 4},
    'C': {'D': 3},
    'D': {'T': 4},
    'E': {'F': 10},
    'F': {'C': 3, 'T': 4},
    'T': {}
}
for k, v in graph.items():
    graph[k] = [(key, value) for key, value in v.items()]
ford_fulkerson = FordFulkerson(graph)
source = 'S'
sink = 'T'

max_flow_value = ford_fulkerson.max_flow(source, sink)
min_cut_edges = ford_fulkerson.min_cut(source)

print("Maximum Flow:", max_flow_value)
print("Min Cut Edges:", min_cut_edges)


Maximum Flow: 11
Min Cut Edges: [('S', 'A'), ('F', 'T'), ('C', 'D')]


In [12]:
# Jesvin
from collections import deque

class Graph:
    def __init__(self, graph):
        self.graph = graph
        self.ROW = len(graph)

    def bfs(self, source, sink, parent):
        visited = [False] * self.ROW
        queue = deque()

        queue.append(source)
        visited[source] = True

        while queue:
            u = queue.popleft()

            for v, capacity in enumerate(self.graph[u]):
                if not visited[v] and capacity > 0:
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u

        return visited[sink]

    def ford_fulkerson(self, source, sink):
        parent = [-1] * self.ROW
        max_flow = 0

        while self.bfs(source, sink, parent):
            path_flow = float("inf")
            s = sink

            # Find the minimum residual capacity of the edges along the path filled by BFS
            while s != source:
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            max_flow += path_flow

            # Update residual capacities and reverse edges along the path
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]

        # Find vertices reachable from source (S) and not reachable (T)
        min_cut_source = []
        for i in range(self.ROW):
            if self.bfs(source, i, parent):
                min_cut_source.append(i)
        
        min_cut_sink = []
        for i in range(self.ROW):
            if i not in min_cut_source:
                min_cut_sink.append(i)
                

        return max_flow, min_cut_source, min_cut_sink

# graph = [
#     [0, 8, 0, 0, 3, 0],
#     [0, 0, 9, 0, 0, 0],
#     [0, 0, 0, 0, 7, 2],
#     [0, 0, 0, 0, 0, 5],
#     [0, 0, 7, 4, 0, 0],
#     [0, 0, 0, 0, 0, 0]
# ]
graph = [[0, 4, 2, 6, 0, 0,0,0],
        [0, 0, 0, 0, 3, 6,0,0],
        [0, 0, 0, 0, 0, 3,0,0],
        [0, 0, 0, 0, 0, 0,10,0],
        [0, 0, 0, 0, 0, 0,0,4],
        [0, 0, 0, 0, 0, 0,0,4],
        [0, 0, 3, 0, 0, 0,0,4],
        [0, 0, 0, 0, 0, 0,0,0]]

g = Graph(graph)
source = 0
sink = len(graph) - 1

max_flow, source_cut, sink_cut = g.ford_fulkerson(source, sink)
print("Max Flow:", max_flow)
print("Min Cut sink:", source_cut)
print("Min Cut sink:", sink_cut)


Max Flow: 11
Min Cut sink: [0, 2, 3, 6]
Min Cut sink: [1, 4, 5, 7]


In [13]:
# J'amie
#### FORD FULKERSON ####

class Graph:

    def __init__(self, graph):
        self.graph = graph
        self.num_vertices = len(graph)


    def FordFulkerson(self, start_node, sink_node): # Returns the maximum flow from s to t in the given graph

      parent = [-1]* self.num_vertices     # This array is filled by BFS and to store path

      max_flow = 0      # There is no flow initially

      while self.BFS(start_node, sink_node, parent) :       # Augment the flow while there is path from source to sink

        path_flow = float("Inf")
        s = sink
        while(s != source):
          path_flow = min (path_flow, self.graph[parent[s]][s])
          s = parent[s]

        max_flow += path_flow       # Add path flow to overall flow

        v = sink
        while(v != source):     # update residual capacities of the edges and reverse edges on the path
          u = parent[v]
          self.graph[u][v] -= path_flow
          self.graph[v][u] += path_flow
          v = parent[v]

      return max_flow

    def BFS(self, start_node, sink_node, parent):           # BFS

      visited = [False]*self.num_vertices # Start with all vertices unvisited
      queue = []
      queue.append(start_node)  # Add start node to queue
      visited[start_node] = True  # Mark the start node as visited

      while queue:
        vertex = queue.pop(0)  # Get a vertex off the top of the queue

        for ind, val in enumerate(self.graph[vertex]):  # Check for a path to sink_node

          if visited[ind] == False and val > 0:
            queue.append(ind)
            visited[ind] = True
            parent[ind] = vertex
            if ind == sink_node:
              return True

      return False  # Return false if no path found





The maximum possible flow is 11 


In [1]:
class Node:

    def __init__(self, v):
        self.value = v
        self.inNeighbors = []
        self.outNeighbors = []
        self.status = "unvisited"
        self.estD = np.inf

    def hasOutNeighbor(self, v):
        if v in self.outNeighbors:
            return True
        return False

    def hasInNeighbor(self, v):
        if v in self.inNeighbors:
            return True
        return False

    def hasNeighbor(self, v):
        if v in self.inNeighbors or v in self.outNeighbors:
            return True
        return False

    def getOutNeighbors(self):
        return self.outNeighbors

    def getInNeighbors(self):
        return self.inNeighbors

    def getOutNeighborsWithWeights(self):
        return self.outNeighbors

    def getInNeighborsWithWeights(self):
        return self.inNeighbors

    def addOutNeighbor(self,v,wt):
        self.outNeighbors.append((v,wt))

    def addInNeighbor(self,v,wt):
        self.inNeighbors.append((v,wt))

    def __str__(self):
        return str(self.value)

#### FORD FULKERSON ####
class Graph:

    def __init__(self, graph):
        self.vertices = len(graph)  #  don't need this, instead just use len(residual_graph)
        self.residual_graph = graph

    def addVertex(self,n):
        self.vertices.append(n)

    def addDiEdge(self, u, v, wt = 1):
        u.addOutNeighbor(v, wt = wt)
        v.addInNeighbor(u, wt = wt)

    def addBiEdge(self, u, v, wt = 1):  # add edges in both directions between u and v
        self.addDiEdge(u, v, wt = wt)
        self.addDiEdge(v, u, wt = wt)

    def FordFulkerson(self, start_node, sink_node): # Returns the maximum flow from s to t in the given graph

      parent = [-1]*(len(self.residual_graph))    # This array is filled by BFS and to store path

      max_flow = 0      # There is no flow initially

      while self.BFS(start_node, sink_node, parent) :       # Augment the flow while there is path from source to sink

        path_flow = float("Inf")
        s = sink
        while(s != source):
          path_flow = min (path_flow, self.residual_graph[parent[s]][s])
          s = parent[s]

        max_flow += path_flow       # Add path flow to overall flow

        v = sink
        while(v != source):     # update residual capacities of the edges and reverse edges on the path
          u = parent[v]
          self.residual_graph[u][v] -= path_flow
          self.residual_graph[v][u] += path_flow
          v = parent[v]

        parent = {}

      return max_flow

    def BFS(self, start_node, sink_node, parent):           # BFS

      visited = [False]*(self.vertices)  # Start with all vertices unvisited
      queue = []
      queue.append(start_node)  # Add start node to queue
      visited[start_node] = True  # Mark the start node as visited

      while queue:
        vertex = queue.pop(0)  # Get a vertex off the top of the queue

        for ind, val in enumerate(self.residual_graph[vertex]):  # Check for a path to sink_node

          if visited[ind] == False and val > 0:
            queue.append(ind)
            visited[ind] = True
            parent[ind] = vertex
            if ind == sink_node:
              return True

      return False  # Return false if no path found


    def getDirEdges(self):
        ret = []
        for v in self.residual_graph:  #######################
            ret += [ [v, u] for u in v.getOutNeighbors ]
        return ret

    def reverseEdge(self,u,v):        # reverse the edge between u and v.  Multiple edges are not supported.
        if u.hasOutNeighbor(v) and v.hasInNeighbor(u):

            if v.hasOutNeighbor(u) and u.hasInNeighbor(v):
                return

            self.addDiEdge(v, u)
            u.outNeighbors.remove(v)
            v.inNeighbors.remove(u)

    def __str__(self):
        ret = "Graph with:\n"
        ret += "\t Vertices:\n\t"
        for v in range(len(self.residual_graph)):
            ret += str(v) + ","
        ret += "\n"
        ret += "\t Edges:\n\t"
        for a,b in self.getDirEdges():
            ret += "(" + str(a) + "," + str(b) + ") "
        ret += "\n"
        return ret

# graph = [[0, 16, 13, 0, 0, 0],
# 		[0, 0, 10, 12, 0, 0],
# 		[0, 4, 0, 0, 14, 0],
# 		[0, 0, 9, 0, 0, 20],
# 		[0, 0, 0, 7, 0, 4],
# 		[0, 0, 0, 0, 0, 0]]

graph = [[0, 4, 2, 6, 0, 0,0,0],
        [0, 0, 0, 0, 3, 6,0,0],
        [0, 0, 0, 0, 0, 3,0,0],
        [0, 0, 0, 0, 0, 0,10,0],
        [0, 0, 0, 0, 0, 0,0,4],
        [0, 0, 0, 0, 0, 0,0,4],
        [0, 0, 3, 0, 0, 0,0,4],
        [0, 0, 0, 0, 0, 0,0,0]]

G = Graph(graph)

source = 0; sink = len(graph)-1
print ("The maximum possible flow is %d " % G.FordFulkerson(source, sink))


The maximum possible flow is 11 


In [8]:
# Damion
class Graph:
    def __init__(self, vertices):
        # Initialize the graph with the given number of vertices
        self.V = vertices
        # Create a 2D list to represent the graph's adjacency matrix
        self.graph = [[0 for _ in range(vertices)] for _ in range(vertices)]
        # Initialize the original graph to store the original structure for min-cut computation
        self.original_graph = [[0 for _ in range(vertices)] for _ in range(vertices)]


    def search_path(self, s, t, parent):
            """
            Use Breadth First Search (BFS) to find a path from source to sink.
            """
            # List to keep track of visited nodes
            visited = [False] * self.V
            # Start BFS from the source node
            queue = [s]
            visited[s] = True

            while queue:
                u = queue.pop(0)
                for ind, val in enumerate(self.graph[u]):
                    # If the node is not visited and there's a valid path
                    if not visited[ind] and val > 0:
                        queue.append(ind)
                        visited[ind] = True
                        parent[ind] = u

            # Return True if a path to the sink is found
            return True if visited[t] else False

    def ford_fulkerson(self, source, sink):
            """
            Main Ford-Fulkerson algorithm to compute max flow.
            """
            # List to store the path
            parent = [-1] * self.V
            max_flow = 0

            # While there's a path from source to sink
            while self.search_path(source, sink, parent):
                # Find the maximum flow through the path found
                path_flow = float('inf')
                s = sink
                while s != source:
                    path_flow = min(path_flow, self.graph[parent[s]][s])
                    s = parent[s]

                # Update capacities and reverse edges along the path
                max_flow += path_flow
                v = sink
                while v != source:
                    u = parent[v]
                    self.graph[u][v] -= path_flow
                    self.graph[v][u] += path_flow
                    v = parent[v]

            # Find the edges that constitute the minimum cut
            min_cut = []
            for i in range(self.V):
                for j in range(self.V):
                    if self.graph[i][j] == 0 and self.original_graph[i][j] > 0:
                        min_cut.append((i, j))

            return max_flow, min_cut


    def add_edge(self, u, v, w):
        """
        Add an edge to the graph.
        """
        self.graph[u][v] = w
        # Update the original graph structure for min-cut computation
        self.original_graph[u][v] = w

# Create a graph given in the above diagram
g = Graph(8)
g.add_edge(0, 1, 4)
g.add_edge(0, 2, 2)
g.add_edge(0, 3, 6)
g.add_edge(1, 4, 3)
g.add_edge(1, 5, 3)
g.add_edge(2, 4, 3)
g.add_edge(3, 6, 10)
g.add_edge(4, 7, 4)
g.add_edge(5, 7, 4)
g.add_edge(6, 7, 4)
g.add_edge(6, 2, 3)

source = 0
sink = 7

max_flow, min_cut = g.ford_fulkerson(source, sink)
print(f"Maximum Flow: {max_flow}")
print(f"Minimum Cut Edges: {min_cut}")






Maximum Flow: 11
Minimum Cut Edges: [(0, 1), (0, 2), (1, 5), (2, 4), (4, 7), (6, 7)]


Cutted Edges:
0 - 1
0 - 2
2 - 5
6 - 7
Minimum Cut Value: 13, Maximum Flow: 11


In [1]:
# David Greg Smith
class FordFulkerson:
    def __init__(self, graph):
        self.graph = graph

    def bfs(self, s, t, parent):
        visited = set()
        queue = [(s, float('inf'))]
        visited.add(s)

        while queue:
            u, capacity = queue.pop(0)
            for v, residual_capacity in self.graph[u].items():
                if v not in visited and residual_capacity > 0:
                    parent[v] = u
                    min_capacity = min(capacity, residual_capacity)
                    if v == t:
                        return min_capacity
                    queue.append((v, min_capacity))
                    visited.add(v)

        return 0

    def max_flow(self, source, sink):
        parent = {}
        max_flow = 0

        while True:
            # Use BFS to find an augmenting path
            augmenting_path_capacity = self.bfs(source, sink, parent)
            if augmenting_path_capacity == 0:
                break

            # Update the residual capacities along the augmenting path
            v = sink
            while v != source:
                u = parent[v]
                self.graph[u][v] -= augmenting_path_capacity
                if v not in self.graph:
                    self.graph[v] = {}
                if u not in self.graph[v]:
                    self.graph[v][u] = 0
                self.graph[v][u] += augmenting_path_capacity
                v = u

            # Add the flow of the current augmenting path to the total flow
            max_flow += augmenting_path_capacity

            # Reset parent dictionary for the next iteration
            parent = {}

        return max_flow
    def min_cut(self, source):
        visited = set()
        stack = [source]
        visited.add(source)

        while stack:
            u = stack.pop()
            for v, residual_capacity in self.graph[u].items():
                if v not in visited and residual_capacity > 0:
                    stack.append(v)
                    visited.add(v)

        source_side = visited
        sink_side = set(self.graph.keys()) - visited

        # Find the edges that cross the minimum cut
        cut_edges = []
        for u in source_side:
            for v, capacity in self.graph[u].items():
                if v in sink_side:
                    cut_edges.append((u, v, capacity))

        return source_side, sink_side, cut_edges

# Example usage:
graph = {
    'S': {'A': 4, 'C': 2, 'E': 6},
    'A': {'B': 3, 'D': 6},
    'B': {'T': 4},
    'C': {'D': 3},
    'D': {'T': 4},
    'E': {'F': 10},
    'F': {'C': 3, 'T': 4},
    'T': {}
}
ff = FordFulkerson(graph)
source = 'S'
sink = 'T'
max_flow = ff.max_flow(source, sink)
print(f"Maximum Flow from {source} to {sink}: {max_flow}")

source_side, sink_side, cut_edges = ff.min_cut(source)
print(f"Minimum Cut:")
print(f"Cut Edges: {cut_edges}")


Maximum Flow from S to T: 11
Minimum Cut:
Cut Edges: [('C', 'D', 0), ('S', 'A', 0), ('F', 'T', 0)]


In [6]:
# Wei Min Voon
from collections import defaultdict 

class Graph: 
	#constructor to initialize the graph
	def __init__(self,graph): 
		self.graph = graph #residual graph
		self.org_graph = [i[:] for i in graph] #make a copy of the original graph
		self.ROW = len(graph) #number of rows in the graph
		self.COL = len(graph[0]) #number of columns in the graph

	#BFS to find augmenting paths
	def BFS(self,s, t, parent): 
		visited =[False]*(self.ROW) 
		queue=[] #initialize queue
		queue.append(s) #start from the source
		visited[s] = True

		while queue: 
			u = queue.pop(0) #dequeue vertex
			#explore adjacent vertices in the residual graph
			for ind, val in enumerate(self.graph[u]): 
				if visited[ind] == False and val > 0 : 
					queue.append(ind) #enqueue unvisited neighbors with available capacity
					visited[ind] = True
					parent[ind] = u  #store the parent vertex for path reconstruction
					
		return True if visited[t] else False
	
	#DFS for transversal and identifying vertices that are reachable from the source
	def dfs(self, graph,s,visited):
		visited[s]=True
		for i in range(len(graph)):
			#recursively visit unvisited neighbors with capacity
			if graph[s][i]>0 and not visited[i]:
				self.dfs(graph,i,visited)

	# Ford-Fulkerson algorithm to find the maximum flow and minimum cut
	def FordFulkerson(self, source, sink):
		parent = [-1] * (self.ROW)
		max_flow = 0 #initialize maximum flow value
		min_cut_value = 0  #initialize minumum cut value

		while self.BFS(source, sink, parent):
			path_flow = float("Inf") #initialize path flow as infinity 
			s = sink
			while s != source:
				path_flow = min(path_flow, self.graph[parent[s]][s])
				s = parent[s] #backtrack to find the bottleneck capacity

			max_flow += path_flow  #add the bottleneck capacity to the maximum flow

			v = sink
			while v != source:
				u = parent[v]
                #update the residual capacities and reverse edges
				self.graph[u][v] -= path_flow
				self.graph[v][u] += path_flow
				v = parent[v]

		visited = len(self.graph) * [False]
		self.dfs(self.graph, source, visited)


        #identify the minimum cut edges and calculate the minimum cut value
		for i in range(self.ROW):
			for j in range(self.COL):
				if self.graph[i][j] == 0 and self.org_graph[i][j] > 0 and visited[i] and not visited[j]:
					print(str(i) + " - " + str(j))
					min_cut_value += self.org_graph[i][j]  #calculate the minimum cut value
		

	
		print("Minimum Cut Value: " + str(min_cut_value) +", Maximum Flow: " + str(max_flow))

#create the graph based on the given question
graph = [[0, 4, 2, 6, 0, 0, 0, 0], #node0 - s(source)
		[0, 0, 0, 3, 6, 0, 0, 0], #node1
		[0, 0, 0, 0, 0, 3, 0, 0], #node2
		[0, 0, 0, 0, 0, 0, 10, 0], #node3
		[0, 0, 0, 0, 0, 0, 0, 4], #node4
		[0, 0, 0, 0, 0, 0, 0, 4], #node5
		[0, 0, 3, 0, 0, 0, 0, 4], #node6
		[0, 0, 0, 0, 0, 0, 0, 0]] #node7 - t(sink)


g = Graph(graph)
source = 0
sink = 7

# print out the output 
print("Cutted Edges:")
g.FordFulkerson(source, sink)

Cutted Edges:
0 - 1
2 - 5
6 - 7
Minimum Cut Value: 11, Maximum Flow: 11
